# Cryptography Project: One-Time and Many-Time Pad Implementations

## Problem 1: Understanding One-Time Pad

### Overview
A one-time pad refers to a cryptographic system where a secret key is generated from a fully, or uniformly, random set of values and used only once to both encrypt and decrypt a single message. If the following conditions are met, then use of a one-time pad enables “perfect secrecy”, which indicates that given the ciphertext of the message, an adversary gains no additional information about the underlying plaintext message, even if the adversary has theoretically infinite computing power:

- The secret key is only used once
- The values used to generate the key must be truly random
- The secret key is securely shared between the sending and receiving parties
- The secret key must be at least as long as the plaintext being encrypted

### Theoretical Foundation
This last requirement is based on a theorem by Claude Shannon, which is stated formally as:
$$\text{Any cipher achieving perfect secrecy requires that } |K| \geq |M|,$$
where $ K $ is the key space and $ M $ is the plaintext message space. Like other encryption systems, one-time pads also adhere to Kerckhoffs's Principle, which states: “[The method of encryption] must not be required to be secret, and it must be able to fall into the enemy’s hands without causing inconvenience.” This principle implies that even if an adversary has knowledge of the encryption technique, they will be unable to decrypt without the secret key.

### Limitations
However, the requirement of a key being at least as long as the plaintext message limits its practical use in many modern applications, including anything related to digital media.

### Sources
- [TechTarget: One-Time Pad Definition](https://www.techtarget.com/searchsecurity/definition/one-time-pad)
- [University of Illinois CS407 Scribe Notes](https://courses.grainger.illinois.edu/CS407/fa2023/Scribe%202.pdf)

## Problem 2: One-Time Pad Implementation

### Introduction
This section details the implementation of a One-Time Pad (OTP) encryption and decryption system, showcasing both Alice's encryption process and Bob's decryption process.

### Alice's Encryption

#### Example Output
```
Enter the plaintext: hello world

--- Encryption Complete ---
Plaintext:         hello world
Key (hex):         7ca8980a42c20dc8f9cdd3
Ciphertext (hex):  14cdf4662de27aa78ba1b7
```

In [ ]:
import secrets

def otp_encrypt(plaintext: bytes, key: bytes) -> bytes:
    """One-Time Pad encryption: ciphertext = plaintext XOR key."""
    return bytes([p ^ k for p, k in zip(plaintext, key)])

# ===Alice's encryption ===
plaintext_str = input("Enter the plaintext: ")
plaintext_bytes = plaintext_str.encode('utf-8')

# Generate random key of the same length
key_bytes = secrets.token_bytes(len(plaintext_bytes))

# Encrypt via OTP (XOR)
ciphertext_bytes = otp_encrypt(plaintext_bytes, key_bytes)

# Convert key and ciphertext to hex for storage
ciphertext_hex = ciphertext_bytes.hex()
key_hex = key_bytes.hex()

# Write to separate files
with open("ciphertext.hex", "w") as f_ct:
    f_ct.write(ciphertext_hex)

with open("key.hex", "w") as f_key:
    f_key.write(key_hex)

print(f"\n--- Encryption Complete ---")
print(f"Plaintext:         {plaintext_str}")
print(f"Key (hex):         {key_hex}")
print(f"Ciphertext (hex):  {ciphertext_hex}")

#### Explanation
One-time pad encryption means that Alice starts with a plaintext message typed by the user. The message is then converted into digital form, typically in bytes. Next, Alice’s program generates a random key that’s exactly as long as the message. This random key is needed for security, because if the key is truly unpredictable, it prevents anyone from guessing the original text. The encryption step uses the XOR (exclusive OR) operation, which compares each bit of the plaintext with the corresponding bit of the key. If the bits are different, it writes a 1; if they are the same, it writes a 0. This produces the ciphertext, which is unreadable unless you have the key. Finally, Alice saves the ciphertext and key in a shared or secure location. If the key stays secret and is never reused, the one-time pad offers nearly perfect security. This approach makes sure that without the key, decryption is impossible.

### Bob's Decryption

In [ ]:
def otp_decrypt(ciphertext: bytes, key: bytes) -> bytes:
    """One-Time Pad decryption: plaintext = ciphertext XOR key."""
    return bytes([c ^ k for c, k in zip(ciphertext, key)])

# ===Bob's decryption===
# Read the hex-encoded key and ciphertext from the files saved above
with open("key.hex", "r") as f_key:
    key_hex = f_key.read().strip()

with open("ciphertext.hex", "r") as f_ct:
    ciphertext_hex = f_ct.read().strip()

# Convert hex to bytes
key_bytes = bytes.fromhex(key_hex)
ciphertext_bytes = bytes.fromhex(ciphertext_hex)

# Decrypt by XOR
decrypted_bytes = otp_decrypt(ciphertext_bytes, key_bytes)

# Convert the decrypted bytes back to string
decrypted_str = decrypted_bytes.decode('utf-8', errors='replace')

print(f"\n--- Decryption Complete ---")
print(f"Decrypted Plaintext: {decrypted_str}")

#### Explanation
While Alice is responsible for encrypting a message with the one-time pad, Bob’s job is to do the reverse process: decryption. He starts by receiving the ciphertext and the key, which are stored here in separate files. He reads them in, converting from hexadecimal format back into bytes, making sure he works with the exact same data used by Alice. Then, Bob performs the XOR operation again, this time using the ciphertext and the key as inputs. Because XOR is its own inverse, applying it once more recovers the original plaintext. If Alice’s key and ciphertext were transmitted correctly, Bob’s decrypted output will match the exact message Alice sent. After that, Bob simply converts the plaintext bytes back into letters, displaying them on his screen. This part of the process shows the importance to keep the key safe, since anyone who has it can completely read the secret messages.

## Problem 3: Implementing Many-Time Pad

### Introduction
This section presents the implementation of a Many-Time Pad encryption system, a variation of the One-Time Pad where a single key is reused across multiple messages, introducing a significant cryptographic weakness. We include an example output, the Python implementation, and an analysis of its vulnerabilities.

### Example Output
```
Enter the plaintext: hello world
Enter the plaintext: my name is alice
Enter the plaintext: are you bob
Enter the plaintext: i hope trudy is not watching this
Enter the plaintext: there need to be 10 lines
Enter the plaintext: and i ran out of text already
Enter the plaintext: but yet i must persevere on
Enter the plaintext: to the depths of mount doom
Enter the plaintext: and so on
Enter the plaintext: and on

--- Encryption Complete ---
Plaintext:         ['68656c6c6f20776f726c64', '6d79206e616d6520697320616c696365', 
                   '61726520796f7520626f62', '6920686f7065207472756479206973206e6f74207761746368696e672074686973', 
                   '7468657265206e65656420746f206265203130206c696e6573', 
                   '616e6420692072616e206f7574206f66207465787420616c7265616479', 
                   '627574207965742069206d75737420706572736576657265206f6e', 
                   '746f2074686520646570746873206f66206d6f756e7420646f6f6d', 
                   '616e6420736f206f6e', '616e64206f6e']
Key (hex):         33f198a8fb270f8e6ff8e43ab52f202a611fe5f9d3fa236bf6ebbbca05e27b0633
Ciphertext (hex):  ['5b94f4c4940778e11d9480', '5e88b8c69a4a6aae068bc45bd946434f', 
                    '5283fd8882487aae0d9786', 
                    '5ad1f0c78b422ffa1d8d80439546530a0f7091d9a49b57089e82d5ad2596136f40', 
                    '4799fdda9e0761eb0a9cc44eda0f424f412ed5d9bf934d0e85', 
                    '529ffc8892077def01d88b4fc10f4f4c416b8081a7da4207848edaae7c', 
                    '5184ec8882427bae06d8894fc65b005a046d969ca59f510ed684d5', 
                    '479eb8dc93422fea0a889052c60f4f4c41728a8cbd8e030f9984d6', 
                    '529ffc8888482fe101', '529ffc889449']
```

In [ ]:
import secrets

def mtp_encrypt(plaintext: bytes, key: bytes) -> bytes:
    """Many-Time Pad encryption: ciphertext = plaintext XOR key."""
    return bytes([p ^ k for p, k in zip(plaintext, key)])
    
many_times = 10
plaintext_bytes = [None] * many_times
plaintext_hex = [None] * many_times
ciphertext_hex = [None] * many_times

# === Alice's encryption ===
for i in range(many_times):
    plaintext_str = input("Enter the plaintext: ")
    plaintext_bytes[i] = plaintext_str.encode('utf-8')
    plaintext_hex[i] = plaintext_bytes[i].hex()

# Generate random key of the max length needed
key_bytes = secrets.token_bytes(max(len(j) for j in plaintext_bytes))
# Convert to hex for storage
key_hex = key_bytes.hex()

# Encrypt via OTP (XOR)
for i in range(many_times):
    ciphertext_bytes = mtp_encrypt(plaintext_bytes[i], key_bytes)
    # Convert to hex for storage
    ciphertext_hex[i] = ciphertext_bytes.hex()

# Write to separate files
with open("plaintext.hex", "w") as f_pt:
    for line in plaintext_hex:
        f_pt.write(" ".join(line) + "\n")

with open("ciphertext.hex", "w") as f_ct:
    for line in ciphertext_hex:
        f_ct.write(" ".join(line) + "\n")

with open("key.hex", "w") as f_key:
    f_key.write(key_hex)

print(f"\n--- Encryption Complete ---")
print(f"Plaintext:         {plaintext_hex}")
print(f"Key (hex):         {key_hex}")
print(f"Ciphertext (hex):  {ciphertext_hex}")

### Analysis of Vulnerabilities
The many-time pad implementation is a direct derivative of the one-time pad. Despite being nearly the same, it creates a major weakness. The only meaningful difference needed was to generate a key suitably long enough for all messages. The plaintext messages are submitted and then a key long enough to encrypt the longest plaintext message is generated. The encryption step uses the XOR operation between the key and the message, in turn creating the ciphertext. The same key is used for all messages; if a message is shorter than the key, then the key and the plaintext apply XOR operation for the length of the plaintext message.

The most obvious result is that plaintext characters (and words) in the same position between messages are encrypted into the same ciphertext. Spaces are especially common and a strong weakness. For example, consider the messages:
- “and i ran out of text already”
- “and so on”
- “and on”

These result in ciphertext (in hex):
- 529ffc8892077def01d88b4fc10f4f4c416b8081a7da4207848edaae7c
- 529ffc8888482fe101
- 529ffc889449

This leads to the understanding that the XOR of the key at any given position yields the same result. If we XOR those two ciphertexts together, it reveals areas of matching words and replicates the same pattern as the same two plaintexts would produce when an XOR is performed on them. When 10 ciphertexts are all analyzed the same way, many insights into the similarities (and differences) between the messages become apparent, and so guessing the contents becomes possible; thus the crib drag is born. Additionally, frequency analysis can help give guesses direction.

## Problem 4: Validating Final Solution and Iterative Process

### Introduction
This section addresses the validation of a decrypted message from a Many-Time Pad system and provides an iterative process for cryptanalysis using crib dragging. It includes two main components: a validation script and an interactive crib dragging tool.

### Validating Final Solution

#### Example Output
```
Final Decrypted Message:
When using a stream ciphernever use the key more than once

Validation Status:
Encryption Valid: True

Re-encrypted Message Hex:
71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986
e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b

Original Target Ciphertext Hex:
71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986
e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b
```

In [ ]:
import sys
import re

# Function to XOR two hexadecimal strings of the same length
def xor_hex_strings(hex1, hex2):
    bytes1 = bytes.fromhex(hex1)
    bytes2 = bytes.fromhex(hex2)
    return ''.join(chr(b1 ^ b2) for b1, b2 in zip(bytes1, bytes2))

# Function to XOR a ciphertext with a crib to derive a key segment
def derive_key_from_crib(ciphertext_hex, crib, position):
    ciphertext_bytes = bytes.fromhex(ciphertext_hex)
    crib_bytes = crib.encode('utf-8')
    key_segment = ''.join(chr(c ^ crib_bytes[i]) for i, c in enumerate(ciphertext_bytes[position:position + len(crib_bytes)]))
    return key_segment

# Function to XOR message bytes with the key to decrypt the message
def xor_with_key(message_bytes, key_bytes):
    return ''.join(chr(b ^ key_bytes[i % len(key_bytes)]) for i, b in enumerate(message_bytes))

# Function to clean a hexadecimal string by removing invalid characters
def sanitize_and_pad_hex_string(hex_string):
    # Keep only valid hexadecimal characters (0-9, a-f, A-F)
    sanitized = ''.join(c for c in hex_string if c in '0123456789abcdefABCDEF')
    # Length is even by truncating the last character if the length is odd
    if len(sanitized) % 2 != 0:
        sanitized = sanitized[:-1]
    return sanitized

# Function to re-encrypt the decrypted message using the derived key
def encrypt_with_key(plaintext, key_bytes):
    plaintext_bytes = plaintext.encode('utf-8')
    return ''.join(f'{b ^ key_bytes[i % len(key_bytes)]:02x}' for i, b in enumerate(plaintext_bytes))

# All 10 provided ciphertexts
ciphertexts = [
    "71felace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba8"
    "5d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e"
    "071ce9d8988adb4afedf46a840",
    "71felace559ale7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf8"
    "1dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504ebal2b22f"
    "5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdblaf5389"
    "719795elf0179cb77c5c456954d88f3",
    "72fe069c51c81a20775928c7879d4fd2a93c3acf3f69fe5fe2e9493a303d9ea9"
    "8c4e5b60ae40al46058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd38"
    "4e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9",
    "67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8"
    "bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2fbd6a291571e40balafd3b4"
    "blfe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd",
    "71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ealece4e3"
    "1d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aalabc39"
    "4f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358"
    "c5c3a421b06",
    "6ef914ce5989152b321a769ad79c42c7be6f6adf2fab19delfc339d84f04ad3a5"
    "89dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab13"
    "81848cfd2d6898918efc046b00b8940bb08e3f313cb23b3dfd8645cfcd80ff824"
    "89",
    "71felace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab8"
    "2cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd36"
    "4605e49ddb83df48eedc42b60c900fb14db4b229ca74b6c4d96442e1c34df8288"
    "f5c3a450a527ecc7c82865b8e",
    "71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a28"
    "5dfa0f01cff101d764983f29bf5",
    "71felace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a49"
    "88ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18"
    "420be0c8cb89924cf3cd5ee448950efd5cabe500c120f2d9d26440ebc34de0298"
    "11977430b01748276d79012955cc6a65aebb9054becda5c9278",
    "71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98"
    "fc5e5f809e20b0c7756b2e08aab83bc5560e257"
]

# Target ciphertext to decrypt
target_message_hex = "71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986"
                     "e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b"

# Sanitize ciphertexts
sanitized_padded_ciphertexts = [sanitize_and_pad_hex_string(ct) for ct in ciphertexts]
sanitized_padded_target_message_hex = sanitize_and_pad_hex_string(target_message_hex)

# Initialize a partial key with null bytes
partial_key = bytearray(len(sanitized_padded_target_message_hex) // 2)

# Apply cribs to construct the partial key
final_message_corrections = {
    "When using a stream cipher": [0],
    "never use the key more than once": [26]
}

for crib, positions in final_message_corrections.items():
    for position in positions:
        key_segment = derive_key_from_crib(sanitized_padded_target_message_hex, crib, position)
        for i, char in enumerate(key_segment):
            partial_key[position + i] = ord(char)

# Decrypt the target message using the fully refined partial key
final_decrypted_message = xor_with_key(bytes.fromhex(sanitized_padded_target_message_hex), partial_key)

# Re-encrypt to verify the decryption
re_encrypted_message_hex = encrypt_with_key(final_decrypted_message, partial_key)

# Compare with the original sanitized target ciphertext
is_encryption_valid = re_encrypted_message_hex == sanitized_padded_target_message_hex

# Output results
print("Final Decrypted Message:")
print(final_decrypted_message)

print("\nValidation Status:")
print("Encryption Valid:", is_encryption_valid)

print("\nRe-encrypted Message Hex:")
print(re_encrypted_message_hex)

print("\nOriginal Target Ciphertext Hex:")
print(sanitized_padded_target_message_hex)

### Iterative Process

In [ ]:
import sys
import re

# --- Utility Functions ---

def xor_hex_strings(hex1, hex2):
    """XOR two hexadecimal strings of the same length and return the resulting string."""
    bytes1 = bytes.fromhex(hex1)
    bytes2 = bytes.fromhex(hex2)
    return ''.join(chr(b1 ^ b2) for b1, b2 in zip(bytes1, bytes2))

def derive_key_from_crib(ciphertext_hex, crib, position):
    """
    XOR a segment of the ciphertext (starting at the given position) with the crib.
    Returns the derived key segment as a string.
    """
    ciphertext_bytes = bytes.fromhex(ciphertext_hex)
    crib_bytes = crib.encode('utf-8')
    key_segment = ''.join(chr(c ^ crib_bytes[i]) for i, c in enumerate(ciphertext_bytes[position:position + len(crib_bytes)]))
    return key_segment

def xor_with_key(message_bytes, key_bytes):
    """
    Decrypt (or encrypt) a message by XORing it with a repeating key.
    Returns the resulting string.
    """
    return ''.join(chr(b ^ key_bytes[i % len(key_bytes)]) for i, b in enumerate(message_bytes))

def sanitize_and_pad_hex_string(hex_string):
    """
    Remove any non-hex characters from the input and ensure that the string length is even.
    """
    sanitized = ''.join(c for c in hex_string if c in '0123456789abcdefABCDEF')
    if len(sanitized) % 2 != 0:
        sanitized = sanitized[:-1]
    return sanitized

def encrypt_with_key(plaintext, key_bytes):
    """
    Encrypt the plaintext using the key by XORing. Returns the result in hexadecimal.
    """
    plaintext_bytes = plaintext.encode('utf-8')
    return ''.join(f'{b ^ key_bytes[i % len(key_bytes)]:02x}' for i, b in enumerate(plaintext_bytes))

# --- Provided Ciphertexts and Target Message ---

ciphertexts = [
    "71felace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba8"
    "5d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e"
    "071ce9d8988adb4afedf46a840",
    "71felace559ale7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf8"
    "1dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504ebal2b22f"
    "5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdblaf5389"
    "719795elf0179cb77c5c456954d88f3",
    "72fe069c51c81a20775928c7879d4fd2a93c3acf3f69fe5fe2e9493a303d9ea9"
    "8c4e5b60ae40al46058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd38"
    "4e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9",
    "67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8"
    "bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2fbd6a291571e40balafd3b4"
    "blfe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd",
    "71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ealece4e3"
    "1d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aalabc39"
    "4f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358"
    "c5c3a421b06",
    "6ef914ce5989152b321a769ad79c42c7be6f6adf2fab19delfc339d84f04ad3a5"
    "89dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab13"
    "81848cfd2d6898918efc046b00b8940bb08e3f313cb23b3dfd8645cfcd80ff824"
    "89",
    "71felace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab8"
    "2cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd36"
    "4605e49ddb83df48eedc42b60c900fb14db4b229ca74b6c4d96442e1c34df8288"
    "f5c3a450a527ecc7c82865b8e",
    "71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a28"
    "5dfa0f01cff101d764983f29bf5",
    "71felace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a49"
    "88ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18"
    "420be0c8cb89924cf3cd5ee448950efd5cabe500c120f2d9d26440ebc34de0298"
    "11977430b01748276d79012955cc6a65aebb9054becda5c9278",
    "71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98"
    "fc5e5f809e20b0c7756b2e08aab83bc5560e257"
]

target_message_hex = "71fe0680149083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd8986"
                     "e1cd2b8ccd9f3f35dff101d241da2eacf9cc8d5123fe5a897efbeda4974b"

# --- Sanitize the Ciphertexts ---

sanitized_padded_ciphertexts = [sanitize_and_pad_hex_string(ct) for ct in ciphertexts]
sanitized_padded_target_message_hex = sanitize_and_pad_hex_string(target_message_hex)

# --- Initialize a Partial Key for the Target Message ---
# We create a key of the same length (in bytes) as the target ciphertext.
partial_key = bytearray(len(sanitized_padded_target_message_hex) // 2)

# --- Interactive Crib Dragging Loop ---
def interactive_crib_drag():
    print("Interactive Crib Dragging for Many-Time Pad Cryptanalysis\n")
    print("The target ciphertext is:")
    print(sanitized_padded_target_message_hex)
    while True:
        print("\nCurrent decryption of target message:")
        decrypted_message = xor_with_key(bytes.fromhex(sanitized_padded_target_message_hex), partial_key)
        print(decrypted_message)
        print("\nCurrent key (in hex; unknown bytes are shown as '00'):")
        print(''.join(f'{b:02x}' for b in partial_key))
        print("\nOptions:")
        print("1. Add a crib (update key segment)")
        print("2. Re-encrypt the decrypted message to validate the key")
        print("3. Exit")
        choice = input("Enter choice (1/2/3): ").strip()

        if choice == '1':
            crib = input("Enter crib (known plaintext segment): ").strip()
            pos_input = input("Enter starting position (byte index) for the crib: ").strip()
            try:
                pos = int(pos_input)
            except ValueError:
                print("Invalid position input; please enter an integer.")
                continue
            # Derive the key segment based on the crib guess
            key_segment = derive_key_from_crib(sanitized_padded_target_message_hex, crib, pos)
            print("\nDerived key segment (as string):", key_segment)
            # Update the partial key with the new segment
            for i, char in enumerate(key_segment):
                if pos + i < len(partial_key):
                    partial_key[pos + i] = ord(char)
                else:
                    print("Warning: Crib exceeds key length; truncating.")
                    break
        elif choice == '2':
            # Decrypt using the current key and then re-encrypt to check consistency.
            final_decrypted_message = xor_with_key(bytes.fromhex(sanitized_padded_target_message_hex), partial_key)
            re_encrypted_message_hex = encrypt_with_key(final_decrypted_message, partial_key)
            is_valid = re_encrypted_message_hex == sanitized_padded_target_message_hex
            print("\nRe-encrypted message hex:")
            print(re_encrypted_message_hex)
            print("\nOriginal target ciphertext hex:")
            print(sanitized_padded_target_message_hex)
            print("\nEncryption Valid:", is_valid)
        elif choice == '3':
            print("Exiting interactive crib dragging.")
            break
        else:
            print("Invalid option. Please choose 1, 2, or 3.")

# --- Main Execution ---
if __name__ == "__main__":
    interactive_crib_drag()

## References
1. “Many Time Pad Attack (XOR),” [Crypto Stack Exchange](https://crypto.stackexchange.com/questions/33673/many-time-pad-attack-xor)
2. “Many Time Pad Attack,” [SecInject Blog](https://secinject.wordpress.com/2015/08/12/many-time-pad-attack/)
3. Allyson English, “Perfect Secrecy: The One-Time Pad,” [Medium](https://medium.com/@allyson.english/perfect-secrecy-the-one-time-pad-4ce82b0b1a64)
4. Colleen Que, “One Time Pad Decryption and Crib Dragging,” [Medium](https://medium.com/@quecolleen/one-time-pad-decryption-and-crib-dragging-22389b824cce)